#### What do we do to the missing value 
1. Drop missing values
2. Fill missing values with test statistic
3. Predict missing values with a machine learning algorithm

In [12]:
import pandas as pd
train = pd.read_csv("../data/train.csv")

This will tell us the total number of non null observations present including the total number of entries.  
Once number of entries isn’t equal to number of non null observations, we can begin to suspect missing values.

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


This will display a summary statistics of all observed features and labels.  
The most important to note here is the min value.  
Once we see -1/0 in an observation like age/height/weight, then we have been able to detect missing value.

In [6]:
train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


This will output the first 5 rows of our data.  
Viewing this will give one a quick view on the presence of NaN/-1/0/blank/? among others.

In [8]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


This will tell us the total number of NaN in or data.  

In [15]:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

##### DROPPING NULL OR MISSING VALUES  
This is the fastest and easiest step to handle missing values. However, it is not generally advised. This method reduces the quality of our model as it reduces sample size because it works by deleting all other observations where any of the variable is missing.

In [ ]:
train.dropna()

##### FILLING MISSING VALUES  
This is the most common method of handling missing values. This is a process whereby missing values are replaced with a test statistic like mean, median or mode of the particular feature the missing value belongs to. One can also specify a forward-fill or back-fill to propagate the next values backward or previous value forward.

In [13]:
# Replace NA values for LoanAmount with mean of all non-null values
mean_value = train['LoanAmount'].mean()
train['LoanAmount'] = train['LoanAmount'].fillna(mean_value)

# For Median
median_value = train['ApplicantIncome'].median()
train['ApplicantIncome'] = train['ApplicantIncome'].fillna(median_value)

Alternative way of filling missing value with test statistic is by using our Imputer method found in sklearn.preprocessing.

In [ ]:
from sklearn.preprocessing import Imputer

# This will look for all columns where we have NaN value and replace the NaN value with specified test statistic.
# for mode we specify strategy='most_frequent'
imp = Imputer(missing_values = 'NaN', strategy ='mean', axis = 0)
imp.fit(train)
train = imp.transform(train)

# Running this won't work if there are columns with non-numeric data

In [ ]:
# For back fill 
train.fillna(method='bfill')

# For forward-fill
train.fillna(method='ffill')

# one can also specify an axis to propagate (1 is for rows and 0 is for columns)
train.fillna(method='bfill', axis=1)

the disadvantage of using mean is that the mean is greatly affected by outliers in our data.  
As a result, if outliers are present in our data, then median will be the best out of the box tool to use.

##### Imputing test statistic within a Pipeline
Data pipelines allow one to transform data from one representation to another through a series of steps. Pipelines allow one to apply and chain intermediate steps of transform to our data. For example, one can fill missing values, pass the output to cross validation and grid search and then fit the model in series of steps chained together where the output of one is the input to another.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis =0)
logreg = LogisticRegression()
steps = [('imputation', imp),('logistic_regression', logreg)]
pipeline = Pipeline(steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state  =42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
pipeline.score(X_test, y_test)

##### PREDICTIVE MODEL FOR HANDLING MISSING DATA   
This is by far one of the best and most efficient method for handling missing data. Depending on the class of data that is missing, one can either use a regression model or classification to predict missing data. This works by turning missing features to labels themselves and now using columns without missing values to predict columns with missing values

The process goes thus:

Call the variable where you have missing values as y.

Split data into sets with missing values and without missing values, name the missing set X_test and the one without missing values X_train and take y (variable or feature where there is missing values) off the second set, naming it y_train.

Use one of classification methods to predict y_pred.

Add it to X_test as your y_test column. Then combine sets together.

The only drawback to this approach is that if there is no correlation between attributes with missing data and other attributes in the data set, then the model will be bias for predicting missing values.

In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/train.csv")

In [6]:
# Create a train and test subset where Y is the variable you want to fill the missing values, 
# and X are the features you want to use to predict
X = df[['ApplicantIncome']]
Ý = df[['LoanAmount']]


In [9]:
loan_amount_is_null = df['LoanAmount'].isna()
test_y = df[]

0       True
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
584    False
585    False
586    False
587    False
588    False
589    False
590    False
591    False
592    False
593    False
594    False
595    False
596    False
597    False
598    False
599    False
600    False
601    False
602    False
603    False
604    False
605     True
606    False
607    False
608    False
609    False
610    False
611    False
612    False
613    False
Name: LoanAmount, Length: 614, dtype: bool